In [1]:
import pandas as pd

In [2]:
riot = pd.read_csv('riot_total_data.csv')

In [3]:
riot.head()

,championName,tier,position,teamId,win,kills,deaths,assists,totalDamageDealtToChampions,visionScore,...,pentaKills,neutralMinionsKilled,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,wardsPlaced,gameMode,gameVersion,gameDuration,gameId
0,Pyke,BRONZE,UTILITY,200,True,2,10,15,12072,83,...,0,0,0,0,10285,40,CLASSIC,14.1.553.2414,1919,6897727003
1,Lux,BRONZE,UTILITY,200,True,3,7,14,14289,28,...,0,0,45620,13811,8999,15,CLASSIC,13.12.514.8523,1903,6541607040
2,Yorick,BRONZE,TOP,100,True,7,4,2,25439,17,...,0,12,32505,8652,3468,10,CLASSIC,13.18.530.4653,1687,6714833991
3,Velkoz,BRONZE,MIDDLE,200,True,8,7,13,53886,38,...,0,0,207595,38772,9989,17,CLASSIC,13.24.547.9214,3347,6889755632
4,Swain,BRONZE,UTILITY,200,False,3,10,2,12788,10,...,0,0,37760,10546,13196,4,CLASSIC,13.19.533.1909,1623,6723550363


In [4]:
riot.shape

(3109532, 43)

In [5]:

convert_riot = riot.sort_values(by=['gameId', 'teamId']).reset_index(drop=True)


gameId로 게임을 구별합니다. player10명을 찾고 이외 데이터는 버리도록 했습니다.

In [6]:
overlab = convert_riot['gameId'].value_counts().reset_index()
overlab = overlab.loc[overlab["count"] == 10]['gameId']
overlab.drop_duplicates()

145       6905166485
146       6905184756
147       6905176146
148       6905178629
149       6905176338
             ...    
299450    6652853243
299451    6710894122
299452    6701933012
299453    6652844613
299454    6711521023
Name: gameId, Length: 299310, dtype: int64

In [7]:
convert_riot = convert_riot.loc[convert_riot['gameId'].isin(overlab)]
convert_riot.head(20)

,championName,tier,position,teamId,win,kills,deaths,assists,totalDamageDealtToChampions,visionScore,...,pentaKills,neutralMinionsKilled,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,wardsPlaced,gameMode,gameVersion,gameDuration,gameId
0,Malzahar,GOLD,MIDDLE,100,True,5,3,3,20429,19,...,0,8,109294,19890,3671,9,CLASSIC,13.6.499.7758,1415,6432690956
1,Leona,GOLD,UTILITY,100,True,0,2,14,4560,36,...,0,4,5948,2184,3233,19,CLASSIC,13.6.499.7758,1415,6432690956
2,Vayne,GOLD,TOP,100,True,5,2,3,16028,22,...,0,4,513,268,1814,8,CLASSIC,13.6.499.7758,1415,6432690956
3,Maokai,GOLD,JUNGLE,100,True,3,1,4,5259,24,...,0,123,119107,4475,2816,4,CLASSIC,13.6.499.7758,1415,6432690956
4,Samira,GOLD,BOTTOM,100,True,9,2,5,15932,16,...,0,4,8789,2535,3008,9,CLASSIC,13.6.499.7758,1415,6432690956
5,Braum,GOLD,UTILITY,200,False,0,5,5,3196,28,...,0,0,6353,2382,5664,10,CLASSIC,13.6.499.7758,1415,6432690956
6,Olaf,GOLD,TOP,200,False,0,2,3,5960,13,...,0,4,0,0,1075,7,CLASSIC,13.6.499.7758,1415,6432690956
7,Chogath,GOLD,MIDDLE,200,False,5,6,0,9341,31,...,0,4,78911,5790,13240,11,CLASSIC,13.6.499.7758,1415,6432690956
8,Varus,GOLD,BOTTOM,200,False,2,4,3,10040,15,...,0,4,7241,2296,4631,10,CLASSIC,13.6.499.7758,1415,6432690956
9,Kayn,GOLD,JUNGLE,200,False,3,5,2,7039,8,...,0,136,993,892,8459,1,CLASSIC,13.6.499.7758,1415,6432690956


In [10]:
# Redefine the function to correctly reshape data for each game into a single row
def reshape_game_data_corrected(group):
    # Flatten the group data by concatenating each player's data into a single row
    reshaped = pd.concat([group.iloc[i] for i in range(len(group))], axis=0)
    # reshaped.index = [f'player_{i + 1}_{col}' for i, col in enumerate(reshaped.index)]
    return reshaped.to_frame().T

In [11]:
# Group the data by 'gameId' and apply the corrected transformation
grouped_data_corrected = convert_riot.groupby('gameId').apply(reshape_game_data_corrected)

# Reset index to clean up the dataframe
reshaped_data_final_corrected = grouped_data_corrected.reset_index(drop=True)

# Show the structure of the transformed data
reshaped_data_final_corrected.info(), reshaped_data_final_corrected.head()
reshaped_data_final_corrected.to_csv('reshaped_data_final_corrected.csv', index=False)

KeyboardInterrupt: 